In [ ]:
import pandas as pd
import csv
def normalize_values(read_file,write_file):
    # Load CSV file
    df = pd.read_csv(read_file, header=None,delimiter=',')

    # # Transpose dataframe to treat first row as values
    df = df.transpose()

    # Normalize values using min-max scaling
    df = (df - df.min()) / (df.max() - df.min())

    # # Transpose dataframe back to original format
    df = df.transpose()

    # Save normalized values to CSV file
    df.to_csv(write_file, header=None, index=None)

MOTION REMOVAL 

In [ ]:
import pandas as pd
from scipy.stats import skew,kurtosis
from ast import literal_eval
import pandas as pd
import matplotlib.pyplot as plt 
from scipy import signal
from scipy.fftpack import rfft, irfft, fftfreq, fft,ifft
from scipy.signal import find_peaks, peak_prominences
from numpy import trapz
from scipy.stats import skew as find_skew
import numpy as np
from scipy.stats import skew

In [ ]:
def motion_removal(data):
    
    Fs = 100;                                                   #Sampling Frequency (Hz)
    Fn = Fs/2;                                                  # Nyquist Frequency (Hz)
    Ws = 0.5/Fn;                                                # Passband Frequency Vector (Normalised)
    Wp = 1.5/Fn;                                                # Stopband Frequency Vector (Normalised)
    Rp =   1;                                                   # Passband Ripple (dB)
    Rs = 150; 
    N, Wn = signal.ellipord(Wp,Ws,Rp,Rs)
    z,p,k = signal.ellip(N,Rp,Rs,Wn,'high',output='zpk')
    sos= signal.zpk2sos(z, p, k)
    y = signal.sosfiltfilt(sos, data)
    return y

In [ ]:
def plotting(ir):
    plt.figure(figsize=(16,4))
    plt.plot(ir,label='ppg')
    plt.show()

In [ ]:
import os

folder_path = '.\ppg_new'
# Loop over all the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a CSV file
    if filename.endswith('.csv'):
        # Do something with the file
        df=pd.read_csv(filename,header=None)
        motion_removal(df)
        file_path = os.path.join(folder_path, filename)
        print(f'Processing file: {file_path}')
        # Perform your desired tasks on the file here

In [164]:
import numpy as np
from tensorflow.keras.layers import Input,Dropout, Dense, Conv1D, BatchNormalization, MaxPooling1D, concatenate, Flatten
from tensorflow.keras.models import Model

# define input layers for CNN1
input1 = Input(shape=(240, 1))

# CNN1
conv1 = Conv1D(20, 3, padding='same', activation='relu')(input1)
bn1 = BatchNormalization()(conv1)
pool1 = MaxPooling1D(2)(bn1)

conv2 = Conv1D(20, 3, padding='same', activation='relu')(pool1)
bn2 = BatchNormalization()(conv2)
pool2 = MaxPooling1D(2)(bn2)

conv3 = Conv1D(20, 3, padding='same', activation='relu')(pool2)
bn3 = BatchNormalization()(conv3)
pool3 = MaxPooling1D(2)(bn3)

# flatten CNN1 output
flatten1 = Flatten()(pool3)

# define input layers for CNN2
input2 = Input(shape=(9, 1))

# CNN2
conv1_nn = Conv1D(20, 3, padding='same', activation='relu')(input2)
bn1_nn = BatchNormalization()(conv1_nn)
pool1_nn = MaxPooling1D(2)(bn1_nn)

conv2_nn = Conv1D(20, 3, padding='same', activation='relu')(pool1_nn)
bn2_nn = BatchNormalization()(conv2_nn)
pool2_nn = MaxPooling1D(2)(bn2_nn)

conv3_nn = Conv1D(20, 3, padding='same', activation='relu')(pool2_nn)
bn3_nn = BatchNormalization()(conv3_nn)
pool3_nn = MaxPooling1D(2)(bn3_nn)

# flatten CNN2 output
flatten2 = Flatten()(pool3_nn)

# define input layer for statistical features
input3 = Input(shape=(10,))

# concatenate all features
concatenated = concatenate([flatten1, flatten2, input3])
flat1=Flatten()(concatenated)
# add Dense layers for classification
dense1 = Dense(500, activation='relu')(flat1)
dropout1 = Dropout(0.5)(dense1)
dense2 = Dense(2, activation='softmax')(dropout1)

# define model
model = Model(inputs=[input1, input2, input3], outputs=dense2)


In [165]:
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

In [166]:
model.summary()

Model: "model_22"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_72 (InputLayer)          [(None, 240, 1)]     0           []                               
                                                                                                  
 input_73 (InputLayer)          [(None, 9, 1)]       0           []                               
                                                                                                  
 conv1d_118 (Conv1D)            (None, 240, 20)      80          ['input_72[0][0]']               
                                                                                                  
 conv1d_121 (Conv1D)            (None, 9, 20)        80          ['input_73[0][0]']               
                                                                                           

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
df=pd.read_csv('./dataset_transoform/PPG/ppg_01.csv',header=None)

def nn_intervals_from_signal(signal,sampling_freq:int):
    dt = 1/sampling_freq # Sampling interval
    peaks, _ = find_peaks(signal, height=0.5*np.max(signal),distance=100)
    peaks_diff=peaks*dt
    nn_intervals = np.diff(peaks_diff)
    return nn_intervals




In [ ]:
def nn_intervals_time_domain_features(input_signal,fs):
    nn_intervals=nn_intervals_from_signal(input_signal,fs)
    nni=np.zeros((9,1))
    
    for i in range(len(nn_intervals)):
        if(i<9):
            nni[i][0]=nn_intervals[i]
    meanNN = np.mean(nn_intervals)
    medianNN = np.median(nn_intervals)
    sdnn = np.std(nn_intervals)
    nn50 = [x>0.05 for x in np.diff(nn_intervals)].count(True)
    pNN50 = nn50 / len(nn_intervals) * 100
    rmssd = np.sqrt(np.mean(np.square(np.diff(nn_intervals))))
    return nni,[meanNN,medianNN,nn50]

In [ ]:
import numpy as np
from scipy import signal as sig
import matplotlib.pyplot as plt



def freq_domain_features_from_signal(input_signal):
    """ 
    params : input_signal -> np array
    params: fs -> int

    VLFa area of VLF (0-0.04hz)
    LFa area of VLF   (0.0.4-0.15hz)
    HFa area of VLF   (0.15-0.4hz)
    TOTALa total area of all band
    VLFh highest power in VLF
    LFh highest power in LF
    HFh highest power in HF
    VLFp VLFa/TOTALa
    LFp LFa/TOTALa
    HFp HFa/TOTALa
    LFn normalized area of LF LFa/(LFa+HFa)
    HFn normalized area of HF HFa/(LFa+HFa)
    LFHF ratio of LFa and HFa

    return : [lf_area,hf_area,vlf_p,lf_p,hf_p,lf_n,lfhf]
    """
    # f = np.linspace(0, fs/2, len(input_signal)//2+1)  # Frequency range (0 to Nyquist)
    # # Apply a Hamming window to the input_signal
    # window = signal.hamming(len(input_signal))
    # input_signal_windowed = input_signal * window

    # # Compute the FFT of the windowed input_signal
    # fft = np.fft.rfft(input_signal_windowed)

    # Compute the power spectral density (PSD)
    input_signal=sig.detrend(input_signal, type='constant')
    f, psd =sig.welch(input_signal, nperseg=1024,window='hamming')

    # Define the frequency bands
    vlf_band = (0.003, 0.04)  # Very low frequency (0.003-0.04 Hz)
    lf_band = (0.04, 0.15)    # Low frequency (0.04-0.15 Hz)
    hf_band = (0.15, 0.4)     # High frequency (0.15-0.4 Hz)

    # Compute the indices corresponding to the frequency bands
    vlf_idx = np.where(np.logical_and(f >= vlf_band[0], f <= vlf_band[1]))[0]
    lf_idx = np.where(np.logical_and(f >= lf_band[0], f <= lf_band[1]))[0]
    hf_idx = np.where(np.logical_and(f >= hf_band[0], f <= hf_band[1]))[0]
    # Compute the area under the PSD curve for each band
    vlf_area = np.trapz(psd[vlf_idx], f[vlf_idx])
    lf_area = np.trapz(psd[lf_idx], f[lf_idx])
    hf_area = np.trapz(psd[hf_idx], f[hf_idx])
    total_area = np.trapz(psd, f)
    # Find the frequency at which the PSD has the highest value for each band
    # vlf_h = f[vlf_idx[np.argmax(psd[vlf_idx])]]
    # lf_h = f[lf_idx[np.argmax(psd[lf_idx])]]
    # hf_h = f[hf_idx[np.argmax(psd[hf_idx])]]

    # Compute the VLFp, LFp, and HFp features
    vlf_p = vlf_area / total_area
    lf_p = lf_area / total_area
    hf_p = hf_area / total_area

    # Compute the LFn and HFn features
    lf_n = lf_area / (lf_area + hf_area)
    hf_n = hf_area / (lf_area + hf_area)

    # Compute the LF/HF ratio (LFHF)
    lfhf = lf_area / hf_area
    #  VLFa, LFa,
    # HFa, TOTALa, VLFp, LFp, HFp
    return [vlf_area,lf_area,hf_area,total_area,vlf_p,lf_p,hf_p]

In [ ]:
def data_builder(input_signal,val,arr):
    nn_intervals,time_domain_features=nn_intervals_time_domain_features(input_signal,128)
    freq_domain_features=freq_domain_features_from_signal(input_signal)
    time_domain_features=time_domain_features+freq_domain_features
    if(val>=5): val_class=[0,1]
    else: val_class=[1,0]
    if(arr>=5): arr_class=[0,1]
    else: arr_class=[1,0]
    return input_signal[:240],nn_intervals,time_domain_features,val_class,arr_class


In [ ]:
import os

infiles=os.listdir('dataset_transoform/GSR/')
tarfiles=os.listdir('dataset/labels/')

signals=np.empty((4800,240,1))
nnis=np.empty((4800,9,1))
stats_features=np.empty((4800,10,1))
valclasses=np.empty((4800,2))
arrclasses=np.empty((4800,2))
ind=0
for file in range(len(infiles)):
    data=np.genfromtxt('dataset_transoform/GSR/'+infiles[file],delimiter=',')
    target=np.genfromtxt('dataset/labels/'+tarfiles[file],delimiter=',')
    print(data.shape)
    print(target.shape)
    for i in range(len(data)):
        for f in range(6):
            signal,nn_intervals,stats_feature,valclass,arrclass=data_builder(data[i][384+1280*f:384+1280*(f+1)],target[i][0],target[i][1])
            signals[ind]=signal.reshape(240,1)
            nnis[ind]=nn_intervals
            # print(nn_intervals)
            stats_features[ind]=np.array(stats_feature).reshape(10,1)
            valclasses[ind]=valclass
            arrclasses[ind]=arrclass
            ind+=1
signals=np.array(signals)
nnis=np.array(nnis)
stats_features=np.array(stats_features)
valclasses=np.array(valclasses)
arrclasses=np.array(valclasses)
print(signals.shape)
print(nnis.shape)
print(stats_features.shape)
print(valclasses.shape)
print(arrclasses.shape)
input=[signals,nnis,stats_features]
# np.array(input).tofile('dat.csv',sep=',')
label_val=valclasses
label_arr=arrclasses
# np.array(label_arr).tofile('labv.csv',sep=',')
# np.array(label_val).tofile('laba.csv',sep=',')






In [168]:
model.fit(input, label_val, epochs=10, batch_size=32,  validation_split=0.2)

Epoch 1/10
120/120 [==============================] - 6s 22ms/step - loss: 1.1547 - accuracy: 0.5029 - val_loss: 0.6836 - val_accuracy: 0.5844
Epoch 2/10
120/120 [==============================] - 2s 20ms/step - loss: 0.7219 - accuracy: 0.5690 - val_loss: 0.7009 - val_accuracy: 0.5156
Epoch 3/10
120/120 [==============================] - 2s 20ms/step - loss: 0.6566 - accuracy: 0.6172 - val_loss: 0.7236 - val_accuracy: 0.5073
Epoch 4/10
120/120 [==============================] - 2s 19ms/step - loss: 0.5904 - accuracy: 0.6812 - val_loss: 0.7320 - val_accuracy: 0.5146
Epoch 5/10
120/120 [==============================] - 2s 18ms/step - loss: 0.5681 - accuracy: 0.6919 - val_loss: 0.7586 - val_accuracy: 0.4865
Epoch 6/10
120/120 [==============================] - 2s 19ms/step - loss: 0.5270 - accuracy: 0.7232 - val_loss: 0.7998 - val_accuracy: 0.5177
Epoch 7/10
120/120 [==============================] - 2s 21ms/step - loss: 0.4979 - accuracy: 0.7378 - val_loss: 0.8030 - val_accuracy: 0.5677

In [ ]:
import numpy as np

# Generate example input data
ppg_data = np.random.randn(1000, 240, 1)
ecg_data = np.random.randn(1000, 9, 1)
stats_data = np.random.randn(1000, 10)

# Combine input data into a tuple
input_data = (ppg_data, ecg_data, stats_data)
# Generate example binary labels
binary_labels = np.random.randint(2, size=(1000,))

# Convert binary labels to one-hot encoding
one_hot_labels = np.zeros((1000, 2))
one_hot_labels[np.arange(1000), binary_labels] = 1

# Assign targets variable
targets = one_hot_labels
# Train the model
# model.fit(input_data, targets, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
targets.shape

In [ ]:
model.predict([signals[4790].reshape(1,240,1),nnis[4790].reshape(1,9,1),stats_features[4790].reshape(1,10,1)])

In [ ]:
label_val[4790]

In [ ]:
model.save('cnn_100ep_arousal.h5')

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense, Dropout, Concatenate, Activation

# Input layers
ppg_input = Input(shape=(240, 1), name='ppg_input')
nn_input = Input(shape=(9, 1), name='nn_input')
stat_input = Input(shape=(10,), name='stat_input')

# First CNN layers
conv1 = Conv1D(20, kernel_size=3, padding='same')(ppg_input)
bn1 = BatchNormalization()(conv1)
relu1 = Activation('relu')(bn1)
pool1 = MaxPooling1D(pool_size=2)(relu1)
conv2 = Conv1D(20, kernel_size=3, padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
relu2 = Activation('relu')(bn2)
pool2 = MaxPooling1D(pool_size=2)(relu2)
conv3 = Conv1D(20, kernel_size=3, padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
relu3 = Activation('relu')(bn3)
pool3 = MaxPooling1D(pool_size=2)(relu3)

# Second CNN layers
conv1_nn = Conv1D(10, kernel_size=3, padding='same')(nn_input)
bn1_nn = BatchNormalization()(conv1_nn)
relu1_nn = Activation('relu')(bn1_nn)
pool1_nn = MaxPooling1D(pool_size=2)(relu1_nn)
conv2_nn = Conv1D(10, kernel_size=3, padding='same')(pool1_nn)
bn2_nn = BatchNormalization()(conv2_nn)
relu2_nn = Activation('relu')(bn2_nn)
pool2_nn = MaxPooling1D(pool_size=2)(relu2_nn)

# Concatenate all features
flatten = Concatenate()([
    Flatten()(pool3),
    Flatten()(pool2_nn),
    stat_input
])

# Dense layers
dense1 = Dense(200, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)
dense2 = Dense(500, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)
output = Dense(2, activation='softmax')(dropout2)

# Define model
model = Model(inputs=[ppg_input, nn_input, stat_input], outputs=output)

# Compile model with L2 regularization
from tensorflow.keras import regularizers
reg = regularizers.l2(0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


In [ ]:
r=model.fit(input,label_val,epochs=20,batch_size=16,validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
from keras.layers import Input, Conv1D, Flatten, concatenate, Dense
from keras.models import Model

# Define input layers
input1 = Input(shape=(240, 1))
input2 = Input(shape=(9, 1))

# First CNN layer for input1
conv1_1 = Conv1D(filters=32, kernel_size=3, activation='relu')(input1)
# Second CNN layer for input1
conv1_2 = Conv1D(filters=64, kernel_size=3, activation='relu')(conv1_1)
# Flatten output from input1
flat1 = Flatten()(conv1_2)

# First CNN layer for input2
conv2_1 = Conv1D(filters=16, kernel_size=3, activation='relu')(input2)
# Second CNN layer for input2
conv2_2 = Conv1D(filters=32, kernel_size=3, activation='relu')(conv2_1)
# Flatten output from input2
flat2 = Flatten()(conv2_2)

# Concatenate feature maps from both inputs
merged = concatenate([flat1, flat2], axis=1)

# Define input for statistical features
input3 = Input(shape=(10,))

# Concatenate statistical features with feature vector
merged2 = concatenate([merged, input3], axis=1)

# Fully connected layer for classification
output = Dense(2, activation='softmax')(merged2)

# Define the model
model = Model(inputs=[input1, input2, input3], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [170]:
## importing lazypredict library
import lazypredict
### importing LazyClassifier for classification problem
from lazypredict.Supervised import LazyClassifier
### importing LazyClassifier for classification problem because here we are solving Classification use case.
from lazypredict.Supervised import LazyClassifier
### importing breast Cancer Dataset from sklearn
from sklearn.datasets import load_breast_cancer
### spliting dataset into training and testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(stats_features.reshape(4800,10),vl , test_size=.5, random_state =123)
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric = None)
### fitting data in LazyClassifier
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
### lets check which model did better on Breast Cancer Dataset
print(models)

100%|██████████| 29/29 [00:06<00:00,  4.28it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score   
Model                                                                           
RandomForestClassifier             0.56               0.55     0.55      0.56  \
XGBClassifier                      0.56               0.55     0.55      0.55   
ExtraTreesClassifier               0.56               0.55     0.55      0.55   
LGBMClassifier                     0.55               0.54     0.54      0.54   
BaggingClassifier                  0.55               0.54     0.54      0.54   
QuadraticDiscriminantAnalysis      0.54               0.53     0.53      0.54   
AdaBoostClassifier                 0.55               0.53     0.53      0.53   
NearestCentroid                    0.54               0.53     0.53      0.53   
ExtraTreeClassifier                0.53               0.53     0.53      0.53   
LinearDiscriminantAnalysis         0.56               0.53     0.53      0.50   
RidgeClassifierCV           

In [ ]:
vl=np.empty((4800))
for i in range(4800):
    vl[i]=valclasses[i][0]